# Import the modules

In [1]:
# import the general modules
import cplex
import sys

# import kbase
import os
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'JOSNYJGASTV5BGELWQTUSATE4TNHZ66U'
kbase = cobrakbase.KBaseAPI(token)

# local ModelSEED database path
modelseed_db_path = '..\..\..\..\ModelSEEDDatabase'

cobrakbase 0.2.8


# SimpleThermo dgbin parameterization

## sans dgbin

In [2]:
# define the example individual model, with integer charges, and associated API media package
model = kbase.get_from_ws('e_coli_core.kb', 94253)
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
model.solver = 'optlang-cplex'

# apply the simple thermodynamic constraints 
%run ../../modelseedpy/fbapkg/simplethermopkg.py
stp = SimpleThermoPkg(model)
stp.build_package({"dgbin": False})

# execute the model
print(model.objective)
model.summary()

Minimize
-59.81*cpd00001_c0_potential - 59.81*cpd00002_c0_potential - 3.547*cpd00003_c0_potential + 3.547*cpd00004_c0_potential - 13.0279*cpd00005_c0_potential + 13.0279*cpd00006_c0_potential + 59.81*cpd00008_c0_potential + 59.81*cpd00009_c0_potential + 3.7478*cpd00010_c0_potential - 2.8328*cpd00020_c0_potential - 3.7478*cpd00022_c0_potential - 4.9414*cpd00023_c0_potential + 4.1182*cpd00024_c0_potential - 1.7867*cpd00032_c0_potential - 0.2557*cpd00053_c0_potential - 0.5191*cpd00061_c0_potential + 59.81*cpd00067_c0_potential - 0.0709*cpd00072_c0_potential - 0.205*cpd00079_c0_potential - 0.8977*cpd00101_c0_potential - 0.129*cpd00102_c0_potential - 1.496*cpd00169_c0_potential - 0.361*cpd00236_c0_potential


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## Constrain the model with dgbin

In [3]:
# define the example individual model, with integer charges, and associated API media package
model = kbase.get_from_ws('e_coli_core.kb', 94253)
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
model.solver = 'optlang-cplex'

stp.clear()
# apply the simple thermodynamic constraints 
stp = SimpleThermoPkg(model)
stp.build_package({"dgbin": True})

# export the constrained LP file 
with open('SimpleThermo_dgbin.lp', 'w') as out:
    out.write(str(model.solver))
    
# execute the model
print(model.objective)
model.summary()

optimal 	 rxn00549_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn05312_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn00247_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn00786_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn08865_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn01200_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn08525_c0
--> ERROR: The simulation lack a solution.
optimal 	 FRD7_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn00545_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn09270_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn00173_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn08288_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn00459_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn00083_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn00097_c0
--> ERROR: The simulation lack a solution.
optimal 	 rxn32421_c0
--> ERR

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# FullThermo infeasibility parameterization

## sans dgbin

In [4]:
# define the example individual model, with integer charges, and associated API media package
model = kbase.get_from_ws('e_coli_core.kb', 94253)
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
model.solver = 'optlang-cplex'

# import the FullThermo package
%run ../../modelseedpy/fbapkg/fullthermopkg.py
ftp = FullThermoPkg(model)
ftp.build_package({"dgbin": False, 'modelseed_path':modelseed_db_path})

# execute the model
print(model.objective)
model.summary()

ValueError: Required argument modelseed_path is missing!

## with dgbin

In [ ]:
# define the example individual model, with integer charges, and associated API media package
model = kbase.get_from_ws('e_coli_core.kb', 94253)
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
model.solver = 'optlang-cplex'

# import the FullThermo package
from modelseedpy.fbapkg import fullthermopkg
ftp = fullthermopkg.FullThermoPkg(model)
ftp.build_package({"dgbin": True, 'modelseed_path':modelseed_db_path})

# export the constrained LP file 
with open('FullThermo_dgbin.lp', 'w') as out:
    out.write(str(model.solver))
    
# execute the model
print(model.objective)
model.summary()

# Brainstorming

In [ ]:
for reaction in model.reactions:
    display(model.solver.status)
    stp.variables['dgbinF'][reaction.id]
dir(stp.variables['dgbinF'][reaction.id].primal)